In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import gym
from gym import spaces
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
from scipy.optimize import minimize

folder_path = os.path.join("stocks", "datasets", "dj30", "raw", "*.csv")
csv_files = glob.glob(folder_path)
data_frames = []

for file in csv_files:
    symbol = os.path.splitext(os.path.basename(file))[0]
    print(f"Processing {symbol} from {file}...")
    df = pd.read_csv(file, parse_dates=['Date'])
    # Use data from 2010 to 2019
    mask = (df['Date'] >= '2010-01-01') & (df['Date'] <= '2019-12-31')
    df = df.loc[mask]
    df = df[['Date', 'Adj Close']].set_index('Date')
    df.rename(columns={'Adj Close': symbol}, inplace=True)
    data_frames.append(df)

merged_df = pd.concat(data_frames, axis=1)
merged_df.sort_index(inplace=True)
train_data = merged_df.loc['2010-01-01':'2016-01-01']
test_data = merged_df.loc['2016-01-01':'2018-01-01']



class PortfolioEnv:
    def __init__(self, price_data, window_obs=60, window_state=120):
        self.price_data = price_data.reset_index(drop=True)
        self.window_obs = window_obs
        self.window_state = window_state
        self.n_assets = price_data.shape[1]
        self.reset()
    
    def reset(self):
        self.current_step = max(self.window_obs, self.window_state)
        self.done = False
        self.portfolio_value = 1.0
        self.history = [self.portfolio_value]
        self.weights = np.ones(self.n_assets) / self.n_assets
        return self._get_state()
    
    def step(self, action):
        self.weights = action
        current_prices = self.price_data.iloc[self.current_step].values
        next_prices = self.price_data.iloc[self.current_step+1].values
        asset_returns = (next_prices / current_prices) - 1
        portfolio_return = np.dot(self.weights, asset_returns)
        self.portfolio_value *= (1 + portfolio_return)
        self.history.append(self.portfolio_value)
        self.current_step += 1
        if self.current_step >= len(self.price_data)-1:
            self.done = True
        reward = portfolio_return
        return self._get_state(), reward, self.done, {}
    
    def _get_state(self):
        obs_data = self.price_data.iloc[self.current_step - self.window_obs:self.current_step]
        obs_returns = np.log(obs_data / obs_data.shift(1)).dropna().values.T
        if obs_returns.shape[1] < self.window_obs:
            pad = np.zeros((self.n_assets, self.window_obs - obs_returns.shape[1]))
            obs_returns = np.hstack((obs_returns, pad))
    
        hist = np.array(self.history)
        if len(hist) < self.window_state + 1:
            state_data = np.log(hist[1:] / hist[:-1])
            state_data = np.pad(state_data, (self.window_state - len(state_data), 0), 'constant')
        else:
            window_hist = hist[-(self.window_state+1):]
            state_data = np.log(window_hist[1:] / window_hist[:-1])
        state_data = state_data[np.newaxis, :] 
        
        obs_tensor = torch.tensor(obs_returns, dtype=torch.float32).unsqueeze(0)  
        state_tensor = torch.tensor(state_data, dtype=torch.float32).unsqueeze(0) 
        return (obs_tensor, state_tensor)
    
    def render(self):
        plt.plot(self.history)
        plt.title("Portfolio Value")
        plt.show()


def generate_actions(n_assets, n_actions=50):
    actions = []
    for _ in range(n_actions):
        w = np.random.rand(n_assets)
        w = w / w.sum()
        actions.append(w)
    return np.array(actions)


n_assets = train_data.shape[1]
actions = generate_actions(n_assets, n_actions=50)


class RLPortfolioManager(nn.Module):
    def __init__(self, obs_channels, obs_length, state_channels, state_length, num_actions):
        super(RLPortfolioManager, self).__init__()
        self.obs_conv = nn.Sequential(
            nn.Conv1d(in_channels=obs_channels, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv1d(32, 32, kernel_size=3, stride=1),
            nn.ReLU()
        )
        conv_obs_length = obs_length - 4  
        self.fc_obs = nn.Linear(32 * conv_obs_length, 128)
        
        self.state_conv = nn.Sequential(
            nn.Conv1d(in_channels=state_channels, out_channels=16, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv1d(16, 16, kernel_size=3, stride=1),
            nn.ReLU()
        )
        conv_state_length = state_length - 4
        self.fc_state = nn.Linear(16 * conv_state_length, 128)
        
        self.fc_combined = nn.Linear(128 + 128, 128)
        self.policy_head = nn.Linear(128, num_actions)
        self.value_head = nn.Linear(128, 1)
        
    def forward(self, observation, state):
        obs_features = self.obs_conv(observation)
        obs_features = obs_features.view(obs_features.size(0), -1)
        obs_features = F.relu(self.fc_obs(obs_features))
        
        state_features = self.state_conv(state)
        state_features = state_features.view(state_features.size(0), -1)
        state_features = F.relu(self.fc_state(state_features))
        
        combined = torch.cat([obs_features, state_features], dim=1)
        combined = F.relu(self.fc_combined(combined))
        policy_logits = self.policy_head(combined)
        value = self.value_head(combined)
        return policy_logits, value

obs_channels = n_assets          
obs_length = 60                   
state_channels = 1                
state_length = 120                
num_actions = actions.shape[0]    

policy_net = RLPortfolioManager(obs_channels, obs_length, state_channels, state_length, num_actions)
optimizer = optim.Adam(policy_net.parameters(), lr=1e-4)


def sample_episode_df(data, episode_length=252):
    """Randomly sample a contiguous episode (approximately one trading year) from historical data."""
    if len(data) <= episode_length:
        return data.copy()
    start_idx = np.random.randint(0, len(data) - episode_length)
    episode_df = data.iloc[start_idx:start_idx + episode_length].copy()
    return episode_df

def train_agent(policy_net, optimizer, actions, train_data, num_episodes=100, window_obs=60, window_state=120, gamma=0.99):
    for ep in range(num_episodes):
        episode_df = sample_episode_df(train_data, episode_length=252)
        env = PortfolioEnv(episode_df, window_obs=window_obs, window_state=window_state)
        state = env.reset()
        
        log_probs = []
        values = []
        rewards = []
        done = False
        
        while not done:
            obs, port_state = state
            policy_logits, value = policy_net(obs, port_state)
            probs = F.softmax(policy_logits, dim=1)
            m = torch.distributions.Categorical(probs)
            action_idx = m.sample()
            log_prob = m.log_prob(action_idx)
            value = value.squeeze(1)
            action = actions[action_idx.item()]
            
            next_state, reward, done, _ = env.step(action)
            log_probs.append(log_prob)
            values.append(value)
            rewards.append(reward)
            state = next_state
        
        R = 0
        returns = []
        for r in reversed(rewards):
            R = r + gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns, dtype=torch.float32)
        
        values = torch.stack(values).squeeze(1)
        log_probs = torch.stack(log_probs)
        
        advantage = returns - values.detach()
        policy_loss = -(log_probs * advantage).mean()
        value_loss = F.mse_loss(values, returns)
        loss = policy_loss + value_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Episode {ep+1}/{num_episodes}, Loss: {loss.item():.4f}, Total Reward: {np.sum(rewards):.4f}")
    
    return policy_net

trained_policy = train_agent(policy_net, optimizer, actions, train_data, num_episodes=200)

In [ ]:
test_data = merged_df.loc['2016-01-01':'2018-01-01'] # As an example, we can loop over all test periods

# MVO Implementation and Backtesting


def calculate_mvo_weights(daily_returns_df, risk_free_rate_daily=0.0):
    """
    Calculates weights for the maximum Sharpe ratio portfolio using historical daily returns.
    Assumes daily_returns_df is a pandas DataFrame (window_size x n_assets).
    """
    n_assets = daily_returns_df.shape[1]
    asset_names = daily_returns_df.columns

    # expected returns (mu) and covariance (sigma) from daily returns
    # Use simple mean and covariance of the provided daily returns window
    mu = daily_returns_df.mean()
    Sigma = daily_returns_df.cov()

    # objective function: negative Sharpe ratio (to minimize)
    def neg_sharpe(weights):
        # daily expected return and daily std dev
        p_ret = np.dot(mu, weights)
        p_var = np.dot(weights.T, np.dot(Sigma, weights))
        p_vol = np.sqrt(p_var) if p_var > 0 else 0 # daily volatility

        if p_vol == 0:
             return -np.inf if p_ret > risk_free_rate_daily else 0

        sharpe = (p_ret - risk_free_rate_daily) / p_vol
        return -sharpe # Minimize negative Sharpe

    # Constraints: Weights sum to 1 (fully invested) / Weights are non-negative
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = tuple((0, 1) for _ in range(n_assets)) 

    # initial guess: equal weights
    init_guess = np.ones(n_assets) / n_assets

    #optimization
    result = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=constraints)

    if not result.success:
        print(f"Warning: MVO optimization failed ({result.message}). Falling back to equal weights.")
        return np.ones(n_assets) / n_assets

    return result.x


def run_mvo_backtest(price_data_full, test_period_data, lookback_window=252):
    """
    Runs the daily rolling MVO backtest on the test_period_data.
    Uses price_data_full for historical lookbacks.
    """
    print(f"\nRunning MVO backtest with {lookback_window}-day rolling window...")
    
    returns_full = price_data_full.pct_change()
    test_dates = test_period_data.index
    n_assets = test_period_data.shape[1]
    asset_names = test_period_data.columns

    mvo_weights_history = pd.DataFrame(index=test_dates, columns=asset_names)
    mvo_portfolio_returns = pd.Series(index=test_dates)

    # iterate through each day in the test period
    for i, current_date in enumerate(test_dates):
        
        try:
            end_loc = returns_full.index.get_loc(current_date) 
            lookback_end_loc = end_loc                
            lookback_start_loc = lookback_end_loc - lookback_window

            if lookback_start_loc < 0:
                 print(f"Warning: Not enough history for lookback window on {current_date}. Using equal weights.")
                 weights_t = np.ones(n_assets) / n_assets
            else:
                hist_returns = returns_full.iloc[lookback_start_loc:lookback_end_loc]
                hist_returns = hist_returns.dropna()
                if len(hist_returns) < n_assets + 1: 
                    print(f"Warning: Not enough non-NaN data points ({len(hist_returns)}) in lookback for {current_date}. Using equal weights.")
                    weights_t = np.ones(n_assets) / n_assets
                else:
                    weights_t = calculate_mvo_weights(hist_returns, risk_free_rate_daily=0.0)

        except KeyError:
             print(f"Warning: Date {current_date} not found in returns_full index. Skipping.")
             weights_t = np.ones(n_assets) / n_assets # Or handle differently

        mvo_weights_history.loc[current_date] = weights_t

        try:
            actual_day_returns = returns_full.loc[current_date].fillna(0) 
            daily_portfolio_return = np.dot(weights_t, actual_day_returns)
            mvo_portfolio_returns.loc[current_date] = daily_portfolio_return
        except KeyError:
            mvo_portfolio_returns.loc[current_date] = 0 

    mvo_portfolio_returns = mvo_portfolio_returns.dropna()

    return mvo_portfolio_returns, mvo_weights_history



# RL Agent testing (Backtest)


def run_rl_backtest(policy_net, actions, test_data, window_obs=60, window_state=120):
    """Runs the trained RL agent on the test data."""
    print("\nRunning RL Agent backtest...")
    policy_net.eval() 
    env = PortfolioEnv(test_data, window_obs=window_obs, window_state=window_state)
    start_offset = max(window_obs, window_state)
    return_dates = test_data.index[start_offset + 1 : ]

    state = env.reset()
    rl_portfolio_returns_list = []
    rl_weights_history_list = []
    steps_taken = 0

    done = False
    while not done:
        with torch.no_grad(): 
            obs, port_state = state
            policy_logits, _ = policy_net(obs, port_state)
            probs = F.softmax(policy_logits, dim=1)
            action_idx = torch.argmax(probs, dim=1).item()
            action_weights = actions[action_idx]

        next_state, reward, done, _ = env.step(action_weights)

        rl_portfolio_returns_list.append(reward)
        rl_weights_history_list.append(action_weights)
        state = next_state
        steps_taken += 1

        if steps_taken > len(test_data):
            print("Warning: Backtest loop exceeded data length.")
            break


    if len(rl_portfolio_returns_list) == len(return_dates):
        rl_portfolio_returns = pd.Series(rl_portfolio_returns_list, index=return_dates)
        rl_weights_history = pd.DataFrame(rl_weights_history_list, index=return_dates, columns=test_data.columns)
    else:
         print(f"Warning: Mismatch in RL return list ({len(rl_portfolio_returns_list)}) and expected dates ({len(return_dates)}). Returning partial results.")
         # mismatch handling
         min_len = min(len(rl_portfolio_returns_list), len(return_dates))
         rl_portfolio_returns = pd.Series(rl_portfolio_returns_list[:min_len], index=return_dates[:min_len])
         rl_weights_history = pd.DataFrame(rl_weights_history_list[:min_len], index=return_dates[:min_len], columns=test_data.columns)


    return rl_portfolio_returns, rl_weights_history



# performance

def calculate_performance_metrics(daily_returns, risk_free_rate_annual=0.0):
    """Calculates standard performance metrics from daily returns Series."""
    if daily_returns.empty or daily_returns.isnull().all():
        return pd.Series({
            'Cumulative Return': 0, 'Annualized Return': 0,
            'Annualized Volatility': 0, 'Sharpe Ratio': np.nan, 'Max Drawdown': 0
        }, dtype=float)

    daily_returns = pd.to_numeric(daily_returns, errors='coerce').replace([np.inf, -np.inf], np.nan).fillna(0)

    cumulative_return = (1 + daily_returns).prod() - 1
    # Calculate annualized return robustly
    if len(daily_returns) > 0:
         annualized_return = ((1 + daily_returns.mean()) ** 252) - 1
    else:
         annualized_return = 0

    annualized_volatility = daily_returns.std() * np.sqrt(252)

    # Sharpe Ratio (using daily returns for precision)
    risk_free_rate_daily = (1 + risk_free_rate_annual)**(1/252) - 1
    if annualized_volatility == 0 or pd.isna(annualized_volatility) or len(daily_returns) == 0:
        sharpe_ratio = np.nan
    else:
        daily_mean_excess_ret = daily_returns.mean() - risk_free_rate_daily
        daily_std = daily_returns.std()
        sharpe_ratio = (daily_mean_excess_ret / daily_std) * np.sqrt(252) if daily_std > 0 else np.nan


    # Max Drawdown
    cum_returns_series = (1 + daily_returns).cumprod()
    peak = cum_returns_series.expanding(min_periods=1).max()
    drawdown = (cum_returns_series / peak) - 1
    max_drawdown = drawdown.min()

    metrics = {
        'Cumulative Return': cumulative_return,
        'Annualized Return': annualized_return,
        'Annualized Volatility': annualized_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown
    }
    return pd.Series(metrics)


# MVO Backtest (using the full data for lookback and testing on test_data)
mvo_returns, mvo_weights = run_mvo_backtest(merged_df, test_data, lookback_window=252)

# RL Backtest
rl_returns, rl_weights = run_rl_backtest(trained_policy, actions, test_data, window_obs=60, window_state=120)

# Metrics 

common_index = mvo_returns.index.intersection(rl_returns.index)
mvo_returns_aligned = mvo_returns.loc[common_index]
rl_returns_aligned = rl_returns.loc[common_index]

print("\nCalculating Performance Metrics on Aligned Data...")
mvo_metrics = calculate_performance_metrics(mvo_returns_aligned)
rl_metrics = calculate_performance_metrics(rl_returns_aligned)

# compare results
comparison_df = pd.DataFrame({'MVO (252d Roll)': mvo_metrics, 'RL Agent': rl_metrics})
print("\nPerformance Comparison (Test Period - Aligned Dates):")
print(comparison_df.applymap(lambda x: f"{x:.4f}" if isinstance(x, (int, float)) else x))


#plotting
plt.figure(figsize=(12, 6))
(1 + mvo_returns_aligned).cumprod().plot(label='MVO Portfolio (252d Roll)')
(1 + rl_returns_aligned).cumprod().plot(label='RL Agent Portfolio')
plt.title(f'Portfolio Value Over Time ({common_index.min().date()} to {common_index.max().date()})')
plt.xlabel('Date')
plt.ylabel('Cumulative Value (Log Scale)')
plt.yscale('log') #
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()